In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib as plt
import seaborn as sns

In [2]:
db_user='root'
db_password='123456'
db_host='localhost'
db_port='3306'
db_name='fixed_asset_db'
engine=create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
print('数据库连接成功')

数据库连接成功


测试一：资产存在性测试（识别幽灵资产）   审计思路：长期没有维护保养记录、但状态仍为‘在用’的资产可能已经丢失或毁坏，但仍在账上

In [5]:
ghost_asset_query="select ar.asset_id,ar.asset_name,ar.purchase_cost,ar.purchase_date,ar.location,max(ml.maintenance_date)as last_maintenance_date from asset_register ar left join maintenance_logs ml on ar.asset_id=ml.asset_id where ar.asset_status='在用' group by ar.asset_id having last_maintenance_date<date_sub(curdate(),interval 2 year) or last_maintenance_date is null;"
ghost_assets=pd.read_sql(ghost_asset_query,engine)
if not ghost_assets.empty:
    print(f"发现{len(ghost_assets)}项幽灵资产：状态为在用，但已超过两年无维护记录")
    print(ghost_assets)

发现297项幽灵资产：状态为在用，但已超过两年无维护记录
     asset_id  purchase_cost purchase_date  location last_maintenance_date
0    FA-10000       12894.84    2021-10-19       台湾省                  None
1    FA-10001      559283.76    2019-11-08       甘肃省            2022-05-08
2    FA-10002       90150.39    2015-08-14    内蒙古自治区            2021-11-14
3    FA-10003      353563.91    2019-07-11   澳门特别行政区                  None
4    FA-10004       30051.79    2022-02-12  新疆维吾尔自治区            2022-04-12
..        ...            ...           ...       ...                   ...
292  FA-10493      662903.89    2015-08-06       浙江省            2016-03-06
293  FA-10494       42448.57    2016-12-01       广东省            2020-04-01
294  FA-10496       76292.89    2019-07-31    内蒙古自治区            2021-07-28
295  FA-10498       60383.37    2024-01-17       江苏省                  None
296  FA-10499       73377.22    2022-12-03       台湾省                  None

[297 rows x 5 columns]


测试二：资产估值准确性测试（验证折旧计算） 审计思路：重新计算样本资产的折旧，与账面记录相比，检查是否存在系统性huo

In [9]:
depreciation_check_query="select ds.asset_id,ds.period,ds.monthly_depreciation as recorded_dep,ar.purchase_cost,ar.useful_life_years,ar.salvage_rate from depreciation_schedule ds join asset_register ar on ds.asset_id=ar.asset_id where ds.period='2024-06';"
dep_check=pd.read_sql(depreciation_check_query,engine)
dep_check['calculate_dep']=round((dep_check['purchase_cost']*(1-dep_check['salvage_rate']))/(dep_check['useful_life_years']*12),2)
dep_errors=dep_check[dep_check['calculate_dep']!=dep_check['recorded_dep']]
if not dep_errors.empty:
    print(f"发现{len(dep_errors)}项资产在2024年6月的折旧计提存在计算错误")
    print(dep_errors)

发现17项资产在2024年6月的折旧计提存在计算错误
     asset_id   period  recorded_dep  purchase_cost  useful_life_years  \
8    FA-10008  2024-06       3287.82      173043.37                  5   
66   FA-10073  2024-06       1042.20      109705.04                 10   
70   FA-10078  2024-06        229.15       24121.02                 10   
72   FA-10080  2024-06       1170.87      123249.26                 10   
74   FA-10083  2024-06         28.94        3046.42                 10   
76   FA-10085  2024-06        346.39       18231.19                  5   
138  FA-10159  2024-06       8249.73     1302589.05                 15   
154  FA-10178  2024-06       2287.42      240780.99                 10   
169  FA-10195  2024-06       4868.41      256232.03                  5   
253  FA-10294  2024-06        241.90       50925.89                 20   
254  FA-10295  2024-06        592.25       62341.78                 10   
332  FA-10382  2024-06         61.31        9679.77                 15   
343  FA-103

测试三：资产减值风险评估（识别闲置资产）  审计思路：长期闲置的资产无法为企业带来经济利益，其价值可能已经发生减损，需计提减值准备

In [11]:
idle_asset_query="select asset_id,asset_name,location,department,purchase_cost,purchase_date from asset_register where asset_status='闲置';"
idle_assets=pd.read_sql(idle_asset_query,engine)
if not idle_assets.empty:
    print(f"发现{idle_assets.shape[0]}项闲置资产，需评估是否存在减值风险")
    print(idle_assets)

发现56项闲置资产，需评估是否存在减值风险
    asset_id asset_name location department  purchase_cost purchase_date
0   FA-10034   运输卡车 #34      天津市    网络有限公司部       35950.34    2022-03-10
1   FA-10036   精密机床 #36      四川省    传媒有限公司部       91544.43    2023-01-31
2   FA-10041   精密机床 #41      陕西省    网络有限公司部       10690.55    2022-01-11
3   FA-10046   办公电脑 #46      台湾省    网络有限公司部      205042.58    2018-02-22
4   FA-10066    服务器 #66     黑龙江省    科技有限公司部      103263.36    2022-06-19
5   FA-10071   办公电脑 #71      河南省    网络有限公司部        4246.90    2023-08-11
6   FA-10084   运输卡车 #84      台湾省    信息有限公司部       44189.84    2018-08-25
7   FA-10098   精密机床 #98     黑龙江省    信息有限公司部        8955.80    2023-08-10
8   FA-10101  精密机床 #101      台湾省    网络有限公司部       72314.42    2022-06-20
9   FA-10103   服务器 #103      陕西省    网络有限公司部       31781.50    2021-10-12
10  FA-10108  办公电脑 #108      天津市    科技有限公司部      185070.90    2017-05-13
11  FA-10111   服务器 #111      广东省    信息有限公司部       81673.71    2016-02-09
12  FA-10114  运输卡车 #114      

测试四：资产处置合规性分析    审计思路：处置过程中的巨大亏损可能意味着资产的账面净值在处置前就已不准确，或处置过程不合规。

In [14]:
disposal_analysis_query="select asset_id,disposal_date,disposal_method,proceeds,net_book_value_at_disposal,net_gain_loss from disposal_records order by net_gain_loss ASC;"
disposal_analysis=pd.read_sql(disposal_analysis_query,engine)
large_loss_disposals=disposal_analysis[disposal_analysis['net_gain_loss']<-50000]
if not large_loss_disposals.empty:
    print(f"发现{large_loss_disposals.shape[0]}项资产处置产生重大亏损，需审查其评估定价及审批流程")
    print(large_loss_disposals)
else:
    print('未发现发生重大亏损的资产处置')

未发现发生重大亏损的资产处置
